# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [ ]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [ ]:
!conda remove libbasic -y
!conda build -q basic/conda/libbasic -c statiskit
!conda install -y -q libbasic --use-local -c statiskit

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [ ]:
%%time
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [ ]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = 'basic/src/py/_basic.cpp',
                             decorator = 'basic/src/py/basic/_basic.py',
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time
wrappers.write()

Here is an example of the generated wrappers.
We here present the wrappers for the `BinomialDistribution` class.

In [ ]:
!pygmentize basic/src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
!conda build -q basic/conda/python-basic -c statiskit
!conda install -y -q python-basic --use-local -c statiskit --force

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0
binomial

In [ ]:
binomial.pmf(0)

In [ ]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error.message
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')